In [1]:
from crim_intervals import *
import pandas as pd
import ast
import matplotlib
from itertools import tee, combinations
import numpy as np

In [2]:
# THIS IS DEV COPY for use with CLOSE/EXACT Matches

import numpy as np

# Converts lists to tuples

def lists_to_tuples_a(el):
    if isinstance(el, list):
        return tuple(el)
    else:
        return el

# Filters for the length of the Presentation Type in the Classifier

def limit_offset_size(array, limit):
    under_limit = np.cumsum(array) <= limit
    return array[: sum(under_limit)]


# Gets the the list of offset differences for each group 

def get_offset_difference_list_a(group):
    # if we do sort values as part of the func call, then we don't need this first line
    group = group.sort_values("start_offset")
    group["next_offset"] = group.start_offset.shift(-1)
    offset_difference_list = (group.next_offset - group.start_offset).dropna().tolist()
    return offset_difference_list

# The classifications are done here

def classify_offsets_a(offset_difference_list):
    """
    Put logic for classifying an offset list here
    """
    #
    offset_difference_list = limit_offset_size(offset_difference_list, 40)



    
    alt_list = offset_difference_list[::2]
    
    
    if len(set(offset_difference_list)) == 1 and len(offset_difference_list) > 1:
        return ("PEN", offset_difference_list)
    elif (len(offset_difference_list) %2 != 0) and (len(set(alt_list)) == 1):
        return ("ID", offset_difference_list)
    elif len(offset_difference_list) >= 1:
        return ("Fuga", offset_difference_list)
    else: 
        return ("Singleton", offset_difference_list)
    

   
# adds predicted type, offsets and entry numbers to the results

def predict_type_a(group):
    offset_differences = get_offset_difference_list_a(group)
    predicted_type, offsets = classify_offsets_a(offset_differences)

    group["predicted_type"] = [predicted_type for i in range(len(group))]
    group["offset_diffs"] = [offsets for i in range(len(group))]
    group["entry_number"] = [i + 1 for i in range(len(group))]

    return group

In [3]:
df = pd.read_csv(('ave_test_set.csv'), index_col=0)
df = df.drop(columns=["ema", "ema_url", "end_measure", "end_beat"])
df.head()


,pattern_generating_match,pattern_matched,piece_title,part,start_measure,start_beat,start_offset,end_offset,note_durations,sum_durs
0,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,[Superius],1,1.0,0.0,24.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
1,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,[Superius],105,3.0,884.0,910.0,"[4.0, 8.0, 4.0, 4.0, 6.0, 2.0]",28.0
2,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Altus,3,1.0,16.0,40.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
3,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Tenor,5,1.0,32.0,56.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0
4,"(4, 1, 2, 2, -3)","[4, 1, 2, 2, -3]",Ave Maria,Bassus,7,1.0,48.0,72.0,"[4.0, 8.0, 4.0, 4.0, 4.0, 8.0]",32.0


In [38]:
# now process the original match data df by:
# sorting by start offset, then group by pattern generating match and applying the function above for prev entry
# also give each group a number
# sort by group number and start offset

df2 = df.sort_values("start_offset")
df2["group_number"] = df2.groupby('pattern_generating_match').ngroup()
df2 = df2.sort_values(['group_number', 'start_offset'])
df2["prev_entry_off"] = df2["start_offset"].shift(1)
df2["next_entry_off"] = df2["start_offset"].shift(-1)
df2["last_group_number"] = df2["group_number"].shift(1)
df2["last_off_diff"] = df2["start_offset"] - df2["prev_entry_off"]
df2["next_off_diff"] = df2["next_entry_off"] - df2["start_offset"]

df2["next_off_diff"] = df2["next_off_diff"].abs()
df2["last_off_diff"] = df2["last_off_diff"].abs()

df2.head(10)

,pattern_generating_match,pattern_matched,piece_title,part,start_measure,start_beat,start_offset,end_offset,note_durations,sum_durs,group_number,prev_entry_off,next_entry_off,last_group_number,last_off_diff,next_off_diff
18,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],19,1.0,144.0,156.0,"[3.0, 1.0, 2.0, 2.0, 4.0, 2.0]",14.0,0,NaN,158.0,NaN,NaN,14.0
19,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],20,4.0,158.0,172.0,"[3.0, 1.0, 4.0, 2.0, 4.0, 4.0]",18.0,0,144.0,216.0,0.0,14.0,58.0
21,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,Tenor,28,1.0,216.0,225.0,"[3.0, 1.0, 1.0, 1.0, 3.0, 1.0]",10.0,0,158.0,672.0,0.0,58.0,456.0
20,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],85,1.0,672.0,688.0,"[4.0, 2.0, 4.0, 2.0, 4.0, 4.0]",20.0,0,216.0,328.0,0.0,456.0,344.0
73,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Tenor,42,1.0,328.0,348.0,"[6.0, 2.0, 4.0, 4.0, 4.0, 2.0]",22.0,1,672.0,328.0,0.0,344.0,0.0
71,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,[Superius],42,1.0,328.0,348.0,"[6.0, 2.0, 4.0, 4.0, 4.0, 4.0]",24.0,1,328.0,586.0,1.0,0.0,258.0
74,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Tenor,74,2.0,586.0,594.0,"[3.0, 1.0, 1.0, 1.0, 2.0, 2.0]",10.0,1,328.0,888.0,1.0,258.0,302.0
72,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Altus,106,1.0,888.0,904.0,"[6.0, 2.0, 2.0, 2.0, 4.0, 4.0]",20.0,1,586.0,756.0,1.0,302.0,132.0
177,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Ave Maria,Altus,95,1.0,756.0,776.0,"[6.0, 2.0, 2.0, 2.0, 8.0, 4.0]",24.0,2,888.0,804.0,1.0,132.0,48.0
178,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Ave Maria,Altus,99,1.0,804.0,824.0,"[6.0, 2.0, 2.0, 2.0, 8.0, 4.0]",24.0,2,756.0,864.0,2.0,48.0,60.0


In [43]:
# above we need to adapt:  col for PREVIOUS OFFset and NEXT OFFset
# so that we can find PARALLEL (=0) Forward Gaps (the distance to NEXT entry), and Backward Gaps (distance to PREVIOUS Entry)
# All Proximate Matches are part of same sub Group (and go to classifier)
# Parallels are part of Sub Group but NOT part of Classified.  We will filter them OUT before classification
# Forward ONLY means the NEXT entry is a GAP.  No problem for Forward ONLY, since these are part of the previous set.
# Backward ONLY means the LAST entry was a GAP.  So these are NEW subgroups, since a new Presentation Type begins
# Forward AND Backward Gaps are SINGLETONS:  We will filter then OUT before Classification
# If an entry is the LAST in a Group and also has a BACKWARD gap it is also a SINGLETON

df2["parallel"] = df2["last_off_diff"] == 0
df2["forward_gapped"] = df2["next_off_diff"] >= 20
df2["back_gapped"] = df2["last_off_diff"] >= 20
df2["singleton"] = (df2['forward_gapped'] == True) & (df2['back_gapped'] == True)
df2["split_group"] = (df2['forward_gapped'] == False) & (df2['back_gapped'] == True)
df2["new_group"] = df2['last_group_number'] != df2['group_number'] 

#now mask out Parallels and Singletons
# df2 = df2[df2["parallel"] != True]
# df2 = df2[df2["singleton"] != True]
df2.head()


,pattern_generating_match,pattern_matched,piece_title,part,start_measure,start_beat,start_offset,end_offset,note_durations,sum_durs,...,next_entry_off,last_group_number,last_off_diff,next_off_diff,parallel,forward_gapped,back_gapped,singleton,split_group,new_group
18,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],19,1.0,144.0,156.0,"[3.0, 1.0, 2.0, 2.0, 4.0, 2.0]",14.0,...,158.0,NaN,NaN,14.0,False,False,False,False,False,True
19,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],20,4.0,158.0,172.0,"[3.0, 1.0, 4.0, 2.0, 4.0, 4.0]",18.0,...,216.0,0.0,14.0,58.0,False,True,False,False,False,False
21,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,Tenor,28,1.0,216.0,225.0,"[3.0, 1.0, 1.0, 1.0, 3.0, 1.0]",10.0,...,672.0,0.0,58.0,456.0,False,True,True,True,False,False
20,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",Ave Maria,[Superius],85,1.0,672.0,688.0,"[4.0, 2.0, 4.0, 2.0, 4.0, 4.0]",20.0,...,328.0,0.0,456.0,344.0,False,True,True,True,False,False
73,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Ave Maria,Tenor,42,1.0,328.0,348.0,"[6.0, 2.0, 4.0, 4.0, 4.0, 2.0]",22.0,...,328.0,0.0,344.0,0.0,False,False,True,False,True,True


In [42]:
df3 = df2.drop(columns=["pattern_matched", "part", "pattern_generating_match", "piece_title", "start_measure", "start_beat", "end_offset", "note_durations", "prev_entry_off", "next_entry_off"])
df3.head(50)

,start_offset,sum_durs,group_number,last_group_number,last_off_diff,next_off_diff,parallel,forward_gapped,back_gapped,singleton,split_group,new_group
18,144.0,14.0,0,NaN,NaN,14.0,False,False,False,False,False,True
19,158.0,18.0,0,0.0,14.0,58.0,False,True,False,False,False,False
21,216.0,10.0,0,0.0,58.0,456.0,False,True,True,True,False,False
20,672.0,20.0,0,0.0,456.0,344.0,False,True,True,True,False,False
73,328.0,22.0,1,0.0,344.0,0.0,False,False,True,False,True,True
71,328.0,24.0,1,1.0,0.0,258.0,True,True,False,False,False,False
74,586.0,10.0,1,1.0,258.0,302.0,False,True,True,True,False,False
72,888.0,20.0,1,1.0,302.0,132.0,False,True,True,True,False,False
177,756.0,24.0,2,1.0,132.0,48.0,False,True,True,True,False,True
178,804.0,24.0,2,2.0,48.0,60.0,False,True,True,True,False,False


In [92]:
counter = 0

# the split-group column tells us if we need to start a new sub-group for that set of matches
# if TRUE, then we need to add a new sub_group_number
# if FALSE, then sub_group is the same
for name, group in df3.groupby("group_number"):
    if df3["split_group"] is False:
        df3["sub_group_id"] = counter   
    else:
        counter = counter + 1
        df3["sub_group_id"] = counter

df3.head(10)


,pattern_generating_match,pattern_matched,part,sum_durs,group_number,last_off_diff,next_off_diff,forward_gapped,back_gapped,split_group,sub_group_id
18,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",[Superius],14.0,0,NaN,14.0,False,False,False,41
19,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",[Superius],18.0,0,14.0,58.0,True,False,True,41
20,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",[Superius],20.0,0,456.0,-344.0,False,True,False,41
73,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Tenor,22.0,1,-344.0,0.0,False,False,False,41
72,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Altus,20.0,1,302.0,-132.0,False,True,False,41
177,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Altus,24.0,2,-132.0,48.0,True,False,True,41
176,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",[Superius],24.0,2,60.0,8.0,False,True,False,41
179,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Tenor,24.0,2,8.0,-660.0,False,False,False,41
204,"(-2, -2, -2, -2, 2)","[-2, -2, -2, -2, 2]",Altus,14.0,3,-660.0,7.0,False,False,False,41
209,"(-2, -2, -2, -2, 2)","[-2, -2, -2, -2, 2]",Tenor,9.0,3,7.0,325.0,True,False,True,41


In [108]:
df3.groupby("pattern_matched").first()


,pattern_generating_match,part,sum_durs,group_number,last_off_diff,next_off_diff,forward_gapped,back_gapped,split_group,subgroup_id
pattern_matched,,,,,,,,,,
"[-2, -2, -2, -2, -2]","(-2, -2, -2, -2, -2)",[Superius],14.0,0,14.0,14.0,False,False,False,1.0
"[-2, -2, -2, -2, -3]","(-2, -2, -2, -2, -3)",Tenor,22.0,1,-344.0,0.0,False,False,False,NaN
"[-2, -2, -2, -2, 1]","(-2, -2, -2, -2, 1)",Altus,24.0,2,-132.0,48.0,True,False,True,2.0
"[-2, -2, -2, -2, 2]","(-2, -2, -2, -2, 2)",Altus,14.0,3,-660.0,7.0,False,False,False,3.0
"[-2, -2, -2, 1, 3]","(-2, -2, -2, 1, 3)",[Superius],18.0,4,-178.0,28.0,True,False,True,5.0
"[-2, -2, -2, 2, -2]","(-2, -2, -2, 2, -2)",[Superius],20.0,5,-992.0,16.0,False,False,False,6.0
"[-2, -2, -2, 2, 2]","(-2, -2, -2, 2, 2)",Bassus,24.0,6,-1070.0,111.0,True,False,True,7.0
"[-2, -2, 1, 3, -2]","(-2, -2, 1, 3, -2)",[Superius],13.0,7,-200.0,28.0,True,False,True,9.0
"[-2, -2, 2, 2, -2]","(-2, -2, 2, 2, -2)",Tenor,27.0,8,-833.0,180.0,True,False,True,10.0


In [100]:
for name, group in df3.groupby("group_number"):
    df3.loc[df3.split_group, "subgroup_id"] = range(1, df3.split_group.sum() + 1)
df3.head(20)

,pattern_generating_match,pattern_matched,part,sum_durs,group_number,last_off_diff,next_off_diff,forward_gapped,back_gapped,split_group,subgroup_id
18,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",[Superius],14.0,0,NaN,14.0,False,False,False,NaN
19,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",[Superius],18.0,0,14.0,58.0,True,False,True,1.0
20,"(-2, -2, -2, -2, -2)","[-2, -2, -2, -2, -2]",[Superius],20.0,0,456.0,-344.0,False,True,False,NaN
73,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Tenor,22.0,1,-344.0,0.0,False,False,False,NaN
72,"(-2, -2, -2, -2, -3)","[-2, -2, -2, -2, -3]",Altus,20.0,1,302.0,-132.0,False,True,False,NaN
177,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Altus,24.0,2,-132.0,48.0,True,False,True,2.0
176,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",[Superius],24.0,2,60.0,8.0,False,True,False,NaN
179,"(-2, -2, -2, -2, 1)","[-2, -2, -2, -2, 1]",Tenor,24.0,2,8.0,-660.0,False,False,False,NaN
204,"(-2, -2, -2, -2, 2)","[-2, -2, -2, -2, 2]",Altus,14.0,3,-660.0,7.0,False,False,False,NaN
209,"(-2, -2, -2, -2, 2)","[-2, -2, -2, -2, 2]",Tenor,9.0,3,7.0,325.0,True,False,True,3.0


In [ ]:

df3.loc[df2.split_group, "new_subgroup_id"] = range(1, df3.split_group.sum() + 1)
# df2["new_subgroup_id"] = df2["new_subgroup_id"].ffill()
df3.head(10)
# df2.loc[df2.split_group, "new_subgroup_id"] = a
# df2


In [ ]:
df2.loc[df2.split_group, "new_subgroup_id"] = range(2, df2.split_group.sum() + 2)

df2["new_subgroup_id"] = df2["new_subgroup_id"].ffill().fillna(1)
df2.head(5)

In [ ]:
# Now we need to interate over each group in order to split the groups at wherever "split_group" is TRUE
# # initalize sub-group counter
sub_counter = 0

# the split-group column tells us if we need to start a new sub-group for that set of matches
# if TRUE, then we need to add a new sub_group_number
# if FALSE, then sub_group is the same
for name, group in df2.groupby("group_number"):
    if df2["split_group"] is False:
        df2["sub_group_id"] = sub_counter   
    else:
        sub_counter = sub_counter + 1
        df2["sub_group_id"] = sub_counter

df2.head(10)



In [ ]:
classified = df.applymap(lists_to_tuples_a).sort_values("start_offset").groupby("pattern_matched").apply(predict_type_a)
classified["group_number"] = classified.groupby('pattern_generating_match').ngroup()
classified = classified[["group_number", "pattern_generating_match", "pattern_matched", "part", "start_measure", "start_beat", "entry_number", "start_offset", "sum_durs", "offset_diffs", "predicted_type"]]

# remove singleton entries

classified.drop(classified[classified['predicted_type'] == "Singleton"].index, inplace = True)

# now classified results, in order by OFFSET and ENTRY NUMBER, but the group numbers can overlap with each other.
classified.head(25)

classified = classified.sort_values(["group_number", "entry_number"])
classified= classified[["group_number", "entry_number", "pattern_matched", "part", "start_measure", "start_beat", "start_offset", "sum_durs", "offset_diffs", "predicted_type"]]
classified["offset_list_length"] = classified["offset_diffs"].apply(len) + 1
classified.head(50)


In [ ]:
classified = classified[classified.entry_number <= classified.offset_list_length]
classified

In [ ]:


classify = df.sort_values("start_offset").groupby("pattern_matched")
for pattern_matched, group in classify: 
    classify["prev_ent_offset"] = classify["start_offset"]
    print(prev_ent_offset) 
    # print(group) 

In [ ]:
df2["split_group"]

In [ ]:
#filter out all the parallel entry items
df3 = df2[df2['parallel_entry'] != True] 
df3




In [ ]:

df3["gapped"] = df3["offset_diff"] >= 20
df3.head()

In [ ]:
sub_group_id = 1
sub_group = sub_group_id + 1
sub_group

In [ ]:
gapped = "True"
sub_group_id = 1
if gapped is False:
    df3["sub_group"] = sub_group_id
else:
    df3["sub_group"] = sub_group_id + 1

sub_group


In [ ]:
# now run the classifer, but on DF from which the Parallel Entries have been removed!

classified2 = df3.applymap(lists_to_tuples_a).sort_values("start_offset").groupby("pattern_matched").apply(predict_type_a)
classified2["group_number"] = classified2.groupby('pattern_generating_match').ngroup()
classified2 = classified2[["group_number", "pattern_generating_match", "pattern_matched", "part", "start_measure", "start_beat", "entry_number", "start_offset", "prev_entry_off","sum_durs", "offset_diffs", "predicted_type"]]

# remove singleton entries

# classified2.drop(classified[classified['predicted_type'] == "Singleton"].index, inplace = True)

# now classified results, in order by OFFSET and ENTRY NUMBER, but the group numbers can overlap with each other.

classified2 = classified2.sort_values(["group_number", "entry_number"])
classified2= classified2[["group_number", "entry_number", "pattern_matched", "part", "start_measure", "start_beat", "start_offset", "prev_entry_off", "sum_durs", "offset_diffs", "predicted_type"]]
# classified2 = classified2[classified2["predicted_type"] == "PEN"]

classified2.head()

In [ ]:
l = [16.0, 16.0, 16.0, 836.0]
for count, value in enumerate(l):
    if value >= 5000:
        split_point = [count]
        res = [l[i : j] for i, j in zip([0] + 
          split_point, split_point + [None])] 
  
# printing result 
print ("The split lists are : " +  str(res)) 

    # print(count, value)

In [ ]:
def group_splitter(classified2):
    od = classified2.offset_diffs
    for count, value in enumerate(od):
        if value >= 5000:
            split_point = [count]
            res = [l[i : j] for i, j in zip([0] + 
                split_point, split_point + [None])] 
  
# printing result 
    print ("The split lists are : " +  str(res)) 
    

    # return str(res))

In [ ]:
group_splitter(classified2)

In [ ]:
print(classified2.offset_diffs)

In [ ]:
gap = classified2["start_offset"] - classified2["prev_entry_off"]
# return df2

classified2["gapped"] = gap >= 80
c3 = classified2[classified2["gapped"] == True]
c3.head(50)
